In [1]:
"""
Reference:

https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb
"""
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load and prepare ds
SYSTEM_PROMPT = """
Responde in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

In [3]:
def extract_xml_answer(text:str):
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text:str):
    if '####' not in text:
        return None
    return text.split('####')[1].strip()

def get_gsm8k_questions(split='train'):
    data = load_dataset('openai/gsm8k', 'main')[split]
    data = data.map(lambda x: {
        'prompt':[
            {'role':'system', 'content': SYSTEM_PROMPT},
            {'role':'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data
print(extract_xml_answer("<answer>123</answer>"))
print(extract_hash_answer('12 #### asas'))

123
asas


In [4]:
dataset = get_gsm8k_questions()
dataset

Dataset({
    features: ['question', 'answer', 'prompt'],
    num_rows: 7473
})

In [5]:
# step = 0
# reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs):
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content'] # [{role:system},{role:user},{role:assistance}]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    if '</answer>' in responses[0] and '<answer>' in responses[0] and '<reasoning>' in responses[0] and '</reasoning>' in responses[0]:
        print('-'*20, f"Question:\n{q}\n", '-'*20, f"Answer:\n{answer[0]}\n", '-'*20, f"Response:\n{responses[0]}\n", '-'*20, f"Extracted:\n{extracted_responses[0]}",'\n\n')
    return [2.0 if r==a else 0.0 for r,a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs):
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs):
    """
    Reward function that checks if the completion has as specific format
    """
    pattern = r"^<resoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]['content'] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs):
    """
    Reward function that checks if the completion has a specific format
    """
    pattern = r"<resoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n"
    responses = [completion[0]['content'] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text):
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count+=0.125
    if text.count("\n</reasoning>\n") == 1:
        count+=0.125
    if text.count("\n<answer>\n") == 1:
        count+=0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count+=0.125
        count-=(len(text.split("\n</answer>")[-1])-1)*0.001
    return count

def xml_count_reward_func(completions, **kwargs):
    contents = [completion[0]['content'] for completion in completions]
    return [count_xml(c) for c in contents]


completions = [
    [{"role": "assistant", "content": "<reasoning>\nThe sum of 1 and 2 is 3, which we multiply by 4 to get 12.\n</reasoning>\n<answer>\n(1 + 2) * 4 = 12\n</answer>"}],
    # [{"role": "assistant", "content": "The sum of 3 and 1 is 4, which we multiply by 2 to get 8. So (3 + 1) * 2 = 8."}],
]

xml_count_reward_func(completions)
# 0.382


[0.382]

In [8]:
from trl import GRPOConfig
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

output_dir="outputs/Qwen-0.5B-GRPO"
run_name="QWEN-0.5B-GRPO-gsm8k"

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = 'cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=16,
    max_prompt_length=256,
    max_completion_length=200,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    log_on_each_node=False,
    use_vllm=False,
    vllm_gpu_memory_utilization=0.3,
    vllm_device='cuda',
    report_to='wandb',
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from trl import GRPOTrainer
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    task_type='CAUSAL_LM'
)


trainer = GRPOTrainer(
    model="Qwen/Qwen2.5-0.5B-Instruct",
    processing_class=tokenizer,
    reward_funcs=[
        xml_count_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func
    ],
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    # peft_config
)

trainer.train()

trainer.save_model(output_dir)

[2025-02-22 12:22:13,978] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/samtang/miniconda3/envs/rl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


INFO 02-22 12:22:14 __init__.py:183] Automatically detected platform cuda.


2025-02-22 12:22:14,607	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: druidlangde (druidlangde-tencent). Use `wandb login --relogin` to force relogin


/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:197: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return self.model.forward(*args, **kwargs)


Step,Training Loss
1,0.000000
2,-0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


-------------------- Question:
The Early Bird Dinner offered dinner meals at half off the menu price if you eat between 2-4 pm.  Curtis ordered the Salisbury Steak that costs $16.00 and Rob ordered the Chicken Fried Steak at $18.00.  If they ate at 3 pm, what was the cost of their total bill?
 -------------------- Answer:
17
 -------------------- Response:
<reasoning>
First, we need to calculate the discounted prices of the Salisbury Steak and the Chicken Fried Steak. The Early Bird Dinner offered half off the menu price for dinner meals eaten between 2-4 pm. 

For the Salisbury Steak, the discounted price would be $16.00 / 2 = $8.00.

For the Chicken Fried Steak, the discounted price would be $18.00 / 2 = $9.00.

Now, we add the discounted prices to get the total bill: $8.00 + $9.00 = $17.00.

Therefore, the total bill for Curtis and Rob at 3 pm is $17.00.

<answer>
The total bill for Curtis and Rob at 3 pm is $17.00.
</answer>
 -------------------- Extracted:
The total bill for Curti

/home/samtang/miniconda3/envs/rl/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:197: FutureWarning: `num_logits_to_keep` is deprecated and will be removed in version 4.50 for `Qwen2ForCausalLM.forward`. Use `logits_to_keep` instead.
  return self.model.forward(*args, **kwargs)


-------------------- Question:
Punger collects baseball cards.  He buys 60 packs of baseball cards. Each pack has 7 cards inside. He wants to put these cards in special pages to protect the cards. Each page can hold 10 cards. How many pages does he need to buy?
 -------------------- Answer:
42
 -------------------- Response:
<reasoning>
Punger needs to buy 60 packs of baseball cards, with each pack having 7 cards. So, he needs to buy a total of 60 packs * 7 cards per pack = 420 cards. Each page can hold 10 cards. Therefore, he needs to buy 420 cards / 10 cards per page = 42 pages.

</reasoning>
<answer>
Punger needs to buy 42 pages.

Answer:
\boxed{42}
 -------------------- Extracted:
Punger needs to buy 42 pages.

Answer:
\boxed{42}
-------------------- Question:
Steve owns a lake shop that sells fish. He has a stock of 200 fish. He sells 50 fish and because his refrigerators stop working a third of the remaining fish become spoiled. A new stock of 200 more fish arrives. How many fish

In [3]:
from vllm import LLM
from transformers import AutoModelForCausalLM, AutoTokenizer

output_dir = 'outputs/Qwen-0.5B-GRPO'
model = AutoModelForCausalLM.from_pretrained(output_dir)
model.to('cuda')

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=896, out_features=896, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=896, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=896, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=896, out_features=128, bias=True)
 

In [4]:
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [10]:
from transformers import TextStreamer
import torch

def generate_with_stream(input_text):
    print(f"\n输入: \n{input_text}")
    print("\n输出:")
    
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=1024,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            streamer=streamer
        )
    
    # 完整结果
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 使用
input_text = "During a race transmitted on television, five advertisements were shown, lasting 3 minutes each. One minute of advertising costs $4000. What was the cost of transmitting these advertisements during the race?"
generate_with_stream(input_text)


输入: 
During a race transmitted on television, five advertisements were shown, lasting 3 minutes each. One minute of advertising costs $4000. What was the cost of transmitting these advertisements during the race?

输出:
race? To determine the total cost of transmitting the advertisements, we need to follow these steps: minute of advertising costs $4000. What was the cost of transmitting these advertisements during the 

1. Calculate the total duration of all the advertisements.
2. Determine the cost per minute of advertising.
3. Multiply the total duration by the cost per minute.

First, let's find the total duration of all the advertisements:
\[
5 \text{ advertisements} \times 3 \text{ minutes/advertisement} = 15 \text{ minutes}
\]

Next, we know that one minute of advertising costs $4000. Therefore, the cost for 15 minutes is:
\[
15 \text{ minutes} \times \$4000/\text{minute} = \$60000
\]

So, the total cost of transmitting these advertisements during the race is:
\[
\boxed{60000}
\]<

In [11]:
s0 = torch.randn(2,3,5)
s1 = torch.randn(2,3,5)

In [13]:
torch.cat([s0,s1], dim=1).shape

torch.Size([2, 6, 5])

In [14]:
torch.stack([s0, s1],dim=1).shape

torch.Size([2, 2, 3, 5])

In [15]:
from torch import nn
# 定义 LSTM
lstm = nn.LSTM(input_size=5, hidden_size=10, num_layers=1, batch_first=True)

# 生成随机输入 (batch_size=2, seq_len=3, input_size=5)
x = torch.randn(2, 3, 5)

# 前向传播
output, (hn, cn) = lstm(x)

print(f"output shape: {output.shape}")  # (2, 3, 10)
print(f"hn shape: {hn.shape}")          # (1, 2, 10)
print(f"cn shape: {cn.shape}")          # (1, 2, 10)

output shape: torch.Size([2, 3, 10])
hn shape: torch.Size([1, 2, 10])
cn shape: torch.Size([1, 2, 10])
